In [1]:
1+1

2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import ImageFolder
import os
from tqdm import tqdm
import sys
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch, gc
import pickle
from itertools import combinations
from torch.utils.data import DataLoader, Dataset
from PIL import Image

from models import Unet, get_default_device, to_device, DeviceDataLoader
os.environ['CUDA_VISIBLE_DEVICES']='0'
torch.manual_seed(0)

In [3]:
rectype=['L2','L1','TV']
rectype_combinations=[] 
rectype_strings=[]
for i in range(len(rectype)):
    for p in combinations(rectype, i+1):  # 2 for pairs, 3 for triplets, etc

        rectype_combinations.append(p)
        rectype_strings.append('_'.join(p))
rectype_combinations
rectype_strings

['L2', 'L1', 'TV', 'L2_L1', 'L2_TV', 'L1_TV', 'L2_L1_TV']

In [4]:
radial_lines=[20,40,60,80,100]
dataset_dir='./BIRN_dataset/'
images_dir=(dataset_dir+'birn_png/')
rec_dirs=[(f"{dataset_dir}birn_pngs_{rl}lines_{rt}/") for rt in rectype for rl in radial_lines]
rec_dirs

['./BIRN_dataset/birn_pngs_20lines_L2/',
 './BIRN_dataset/birn_pngs_40lines_L2/',
 './BIRN_dataset/birn_pngs_60lines_L2/',
 './BIRN_dataset/birn_pngs_80lines_L2/',
 './BIRN_dataset/birn_pngs_100lines_L2/',
 './BIRN_dataset/birn_pngs_20lines_L1/',
 './BIRN_dataset/birn_pngs_40lines_L1/',
 './BIRN_dataset/birn_pngs_60lines_L1/',
 './BIRN_dataset/birn_pngs_80lines_L1/',
 './BIRN_dataset/birn_pngs_100lines_L1/',
 './BIRN_dataset/birn_pngs_20lines_TV/',
 './BIRN_dataset/birn_pngs_40lines_TV/',
 './BIRN_dataset/birn_pngs_60lines_TV/',
 './BIRN_dataset/birn_pngs_80lines_TV/',
 './BIRN_dataset/birn_pngs_100lines_TV/']

In [5]:

class OriginalReconstructionDataset(Dataset):
    def __init__(self, radial_line, rec_type_str, datasets_dir, indexes = None, img_size=(256,256)):
        rec_type=rec_type_str.split('_')
        self.images_dir=(dataset_dir+'birn_png/')
        rec_dirs=[(f"{dataset_dir}birn_pngs_{rl}lines_{rt}/") for rt in rectype for rl in radial_lines]
        
        self.rec_images_dirs=[]
        for dir in rec_dirs:
            for rt in rec_type:
                if rt in dir:
                    if str(radial_line) in dir:
                        self.rec_images_dirs.append(dir)
                        break

        self.images = [f for f in os.listdir(self.images_dir) if f.endswith('.png')]
        if indexes is not None:
            self.images = [self.images[i] for i in indexes] 
        self.transform = transforms.Compose([
                        transforms.Grayscale(num_output_channels=1),         
                        transforms.Resize(img_size),
                        #transforms.Lambda(lambda x: x/255.0),
                        transforms.ToTensor()
                        ])
        self.rec_types=rec_type
        self.radial_line=radial_line
        print(self.images_dir)
        print(self.rec_images_dirs)
        print(self.rec_types)
        print(self.radial_line)
    def __len__(self):
    # return length of image samples    
        return len(self.images)

    def __getitem__(self, idx):
        img_name=self.images[idx]
        img = Image.open(self.images_dir+img_name)
        img=self.transform(img)
        rec_imgs=[]
        for rec,dir in zip(self.rec_types,self.rec_images_dirs):
            noisy_name=img_name[:-14]+rec+f'_{self.radial_line}lines.png'            
            tensor=self.transform(Image.open(dir+noisy_name))
            rec_imgs.append(tensor)
        noisy=torch.stack(rec_imgs)
        noisy=torch.squeeze(noisy, 1)
        return (img,noisy)

def first_element(test_dataset):
    for data in test_dataset:
        print(data[0].shape)
        print(data[1].shape)
        break


In [6]:
idx_file='indexes.pkl'
if not os.path.exists(idx_file):
    np.random.seed(seed=42)
    all_indexes=np.random.permutation(len([f for f in os.listdir(images_dir) if f.endswith('.png')]))
    m = len(all_indexes)
    m_train=int(m*0.8)
    m_val = int(m*0.1)
    train_indexes=all_indexes[:m_train]
    val_indexes=all_indexes[m_train:m_train+m_val]
    test_indexes=all_indexes[m_train+m_val:]
    
    with open(idx_file, 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([train_indexes, val_indexes, test_indexes], f)
else:
    with open(idx_file,'rb') as f:  # Python 3: open(..., 'rb')
        train_indexes, val_indexes, test_indexes = pickle.load(f)




In [7]:
#nao precisa dessa celula
batch_size=4
device=get_default_device()
radial_lines=[20,40,60,80,100]

train_dataset={}
train_loaders={}
for rt in rectype_strings:
    for rl in radial_lines:
        train_ds=OriginalReconstructionDataset(rl, rt, dataset_dir, train_indexes)
        train_dataset[rl,rt]=train_ds
        train_loaders[rl,rt]=DeviceDataLoader(torch.utils.data.DataLoader(train_ds, batch_size=batch_size), device)
        first_element(train_ds)


val_dataset={}
val_loaders={}
for rt in rectype_strings:
    for rl in radial_lines:
        val_ds=OriginalReconstructionDataset(rl, rt, dataset_dir, val_indexes)
        val_dataset[rl,rt]=val_ds
        val_loaders[rl,rt]=DeviceDataLoader(torch.utils.data.DataLoader(val_ds, batch_size=batch_size), device)
        first_element(val_ds)

test_dataset={}
test_loaders={}
for rt in rectype_strings:
    for rl in radial_lines:
        test_ds=OriginalReconstructionDataset(rl, rt, dataset_dir, test_indexes)
        test_dataset[rl,rt]=test_ds
        test_loaders[rl,rt]=DeviceDataLoader(torch.utils.data.DataLoader(test_ds, batch_size=batch_size,shuffle=True), device)
        first_element(test_ds)





./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_20lines_L2/']
['L2']
20
torch.Size([1, 256, 256])
torch.Size([1, 256, 256])
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_40lines_L2/']
['L2']
40
torch.Size([1, 256, 256])
torch.Size([1, 256, 256])
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_60lines_L2/']
['L2']
60
torch.Size([1, 256, 256])
torch.Size([1, 256, 256])
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_80lines_L2/']
['L2']
80
torch.Size([1, 256, 256])
torch.Size([1, 256, 256])
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_100lines_L2/']
['L2']
100
torch.Size([1, 256, 256])
torch.Size([1, 256, 256])
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_20lines_L1/']
['L1']
20
torch.Size([1, 256, 256])
torch.Size([1, 256, 256])
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_40lines_L1/']
['L1']
40
torch.Size([1, 256, 256])
torch.Size([1, 256, 256])
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_60lines_L1/']
['L1']
60
torch.Size([1, 256, 256])


In [8]:


## Training function
def train_epoch_den(model, device, dataloader, loss_fn, optimizer, scheduler):
    # Set train mode
    model.train()    
    train_loss = []
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch, image_noisy in tqdm(dataloader): # with "_" we just ignore the labels (the second element of the dataloader tuple)
        image_noisy.to(device)
        result = model(image_noisy)

        # Evaluate loss
        loss = loss_fn(result, image_batch)
        # Backward pass

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
       
             
        # Print batch loss
        #print('\t partial train loss (single batch): %f' % (loss.data))
        train_loss.append(loss.detach().cpu().numpy())

    return np.mean(train_loss)

### Testing function
def test_epoch_den(model, device, dataloader, loss_fn):
    # Set evaluation mode
    model.eval()
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for image_batch, image_noisy in dataloader:
            result = model(image_noisy)
            # Append the network output and the original image to the lists
            conc_out.append(result.cpu())
            conc_label.append(image_batch.cpu())
        # Create a single tensor with all the values in the lists
        conc_out = torch.cat(conc_out)
        conc_label = torch.cat(conc_label) 
        # Evaluate global loss
        val_loss = loss_fn(conc_out, conc_label)
    return val_loss.data

def plot_ae_outputs_den(model,n=10):
    plt.figure(figsize=(21,6))
    for i in range(n):

      ax = plt.subplot(3,n,i+1)
      img = test_dataset[4*i][0].unsqueeze(0)
      image_noisy = test_dataset[4*i][1].unsqueeze(0)
      
      model.eval()

      with torch.no_grad():
         rec_img  = model(image_noisy)

      plt.imshow(img.cpu().squeeze().numpy()[0,:,:], cmap='gist_gray')
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)  
      if i == n//2:
        ax.set_title('Original images')
      ax = plt.subplot(3, n, i + 1 + n)
      plt.imshow(image_noisy.cpu().squeeze().numpy()[0,:,:], cmap='gist_gray')
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)  
      if i == n//2:
        ax.set_title('Corrupted images')

      ax = plt.subplot(3, n, i + 1 + n + n)
      plt.imshow(rec_img.cpu().squeeze().numpy()[0,:,:], cmap='gist_gray')  
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)  
      if i == n//2:
         ax.set_title('Reconstructed images')
    plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.7, 
                    top=0.9, 
                    wspace=0.3, 
                    hspace=0.3)   
    
    plt.savefig('images_256_CS_TV.png')
    plt.show()



In [9]:
def define_optimizer(lr_type, lr, params_to_optimize,max_epochs=500):
    if lr_type == 'constant':
        optimizer = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-05)
        scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
    if lr_type =='exp':
        optimizer  = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-05)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.000001**(1/max_epochs), last_epoch=- 1, verbose=False)
    elif lr_type == 'plateau':
        optimizer = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-05)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1**(1/2), patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    else:
        optimizer = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-05)
        scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer)
    return (lr_type, optimizer,scheduler)
    
    
def save_model(path, epoch, model, optimizer, scheduler, train_loss, val_loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            }, path)
            
def optimizer_to(optim, device):
    for param in optim.state.values():
        # Not sure there are any global tensors in the state dict
        if isinstance(param, torch.Tensor):
            param.data = param.data.to(device)
            if param._grad is not None:
                param._grad.data = param._grad.data.to(device)
        elif isinstance(param, dict):
            for subparam in param.values():
                if isinstance(subparam, torch.Tensor):
                    subparam.data = subparam.data.to(device)
                    if subparam._grad is not None:
                        subparam._grad.data = subparam._grad.data.to(device)

def load_model(model_name, rec_type='', lr_type='constant', learning_rate=1e-4, path='', device='cuda:0'):
    if model_name=='Unet':
        model=Unet(num_inputs=len(rec_type.split('_'))) #1 a 3 canais
    elif model_name=='Resnet_Unet':
        from models import ResnetUnet
        model = ResnetUnet(in_channels=len(rec_type.split('_')))        
        #IF TO ADD NEW MODEL IMPLEMENT H#RE
    else:
        raise Exception('Model not found...')
    
    params_to_optimize = [{'params': model.parameters()}]        
    lr_type, optimizer,scheduler=define_optimizer(lr_type, learning_rate, params_to_optimize)    
    epoch = 0
    train_loss=float('inf')
    val_loss=float('inf')
    
    if path!='':
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        epoch = checkpoint['epoch']
        train_loss = checkpoint['train_loss']
        val_loss = checkpoint['val_loss']
    model.to(device)
    optimizer_to(optimizer,device)
    return (model, optimizer, scheduler, epoch, train_loss, val_loss)

In [10]:
def check_csv(csv_filename):
    if not os.path.exists(csv_filename):
        df_history = pd.DataFrame(columns = ['epoch','lr_scheduler','learning_rate','train_loss','val_loss', 'checkpoint', 'status'])
        last_epoch=0
        last_checkpoint=''
        return df_history, last_epoch, last_checkpoint
    
    folder=csv_filename[:csv_filename.rfind('/')]
    #preparing the csv
    df_history=pd.read_csv(csv_filename)
    df_history = df_history.loc[:, ~df_history.columns.str.contains('^Unnamed')]
    df_history=df_history.drop_duplicates(subset=['epoch'], keep='first')
    
    #find best model (lowest validation loss)
    val_sorted_df=df_history.sort_values(by=['val_loss'])
    for i, row in val_sorted_df.iterrows():
        if os.path.isfile(row['checkpoint']):
            best_epoch=row['epoch']
            best_checkpoint=row['checkpoint']
            best_checkpoint=best_checkpoint.replace('//','/')
            break
    
    #find last saved model
    epoch_sorted_df=df_history.sort_values(by=['epoch'], ascending=False)
    for i, row in epoch_sorted_df.iterrows():
        if os.path.isfile(row['checkpoint']):
            last_epoch=row['epoch']
            last_checkpoint=row['checkpoint']
            last_checkpoint=last_checkpoint.replace('//','/')
            break

    
    #erase rows that do not have a saved model
    df_history=df_history[df_history['epoch']<=last_epoch]

    files_2_delete=[folder+f for f in os.listdir(folder) if f.endswith('.pth')]
    files_2_delete.remove(best_checkpoint)
    files_2_delete.remove(last_checkpoint)    

    for f in files_2_delete:
        #os.remove(f)
        pass
    
    return df_history, last_epoch, last_checkpoint

import pandas as pd

def train(rec_type, radial_lines, model_name, loss_fn, lr_type, learning_rate, dl_train, dl_val,device,max_epochs):

    
    path=f'saved_models/{model_name}_{radial_lines}lines_{rec_type}/'
    if not os.path.exists(path):
        os.makedirs(path)
    
    csv_filename= f'{path}/loss_history.csv'
    df_history, last_epoch, last_checkpoint = check_csv(csv_filename)

    (model, optimizer, scheduler, epoch, train_loss, val_loss) = load_model(model_name,  rec_type, lr_type, learning_rate, path=last_checkpoint, device=device)

    for epoch in range(epoch,max_epochs):

        print('EPOCH %d/%d' % (epoch, max_epochs))        

        ### Training (use the training function)
        train_loss=train_epoch_den(
            model=model, 
            device=device, 
            dataloader=dl_train, 
            loss_fn=loss_fn, 
            optimizer=optimizer, 
            scheduler=scheduler)

        ### Validation  (use the testing function)
        val_loss = test_epoch_den(
            model=model, 
            device=device, 
            dataloader=dl_val,
            loss_fn=loss_fn)
        # Print Validationloss


        train_loss=train_loss if not torch.is_tensor(train_loss) else train_loss.cpu().detach().numpy()
        val_loss=val_loss if not torch.is_tensor(val_loss) else val_loss.cpu().detach().numpy()

        filename=f'{path}/{model.name()}_{radial_lines}lines_{rec_type}_epoch{epoch}_lr_{lr_type}_{optimizer.param_groups[0]["lr"]}.pth'        
        save_model(filename, epoch, model, optimizer, scheduler, train_loss, val_loss)
        
        if val_loss<=df_history['val_loss'].min():
            save_model(filename, epoch, model, optimizer, scheduler, train_loss, val_loss)
            status='new best'
            df_history.replace(to_replace="new best", value='old best')
            best_model=filename
        else:
            status='never best'    
            
        df_history = pd.concat([df_history,
                                pd.DataFrame({'epoch':[epoch],
                                                'lr_scheduler':[lr_type],
                                                'learning_rate':[optimizer.param_groups[0]["lr"]],
                                                'train_loss':[train_loss],
                                                'val_loss':[val_loss], 
                                                'checkpoint':[filename], 
                                                'status':[status]})])
        df_history = df_history.loc[:, ~df_history.columns.str.contains('^Unnamed')]
        
        df_history.to_csv(csv_filename)       
        last_model=filename       


    return df_history




In [11]:
%%time
import mlflow
from sklearn.model_selection import ParameterGrid

def execute(num_radial_lines, data_train, data_val, data_test, device, exp_params):
    experiment_name=f"MRIREC_{num_radial_lines}"
    run_params = {"description":f"Reconstruction using {num_radial_lines} radial lines",
              "tags":{'release.version':'1.0.0'}}
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if not experiment:
        experiment_id=mlflow.create_experiment(experiment_name)
    experiment = mlflow.set_experiment(experiment_name)
    
    run_params.update({"experiment_id": experiment.experiment_id})
    
    print("Experiment_id: {}".format(experiment.experiment_id))
    print("Localização dos artefatos: {}".format(experiment.artifact_location))
    print("Tags: {}".format(experiment.tags))
    print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

    grid_exp = ParameterGrid(exp_params)

    for p_model in grid_exp:
        with mlflow.start_run(**run_params) as run:

            mlflow.log_params({'model': p_model['model'], 'rectype': p_model['rectype'], 'max_epochs': p_model['max_epochs'], 'learning_rate': p_model['learning_rate'], 'batch_size': p_model['batch_size']})

            print(p_model)
            model = p_model['model']
            rectype = p_model['rectype']
            num_channels = len(rectype.split('_'))
            epochs = p_model['max_epochs']
            lr_type, learning_rate = p_model['learning_rate']
            batch_size = p_model['batch_size']

            train_ds= OriginalReconstructionDataset(num_radial_lines, rectype, dataset_dir, train_indexes)
            train_dl= DeviceDataLoader(torch.utils.data.DataLoader(train_ds, batch_size=batch_size), device)
            val_ds=OriginalReconstructionDataset(num_radial_lines, rectype, dataset_dir, test_indexes)
            val_dl= DeviceDataLoader(torch.utils.data.DataLoader(train_ds, batch_size=batch_size), device)

            #train loop:
            history=train(rectype, num_radial_lines, model, torch.nn.MSELoss(), lr_type, learning_rate, train_dl, val_dl, device,epochs)
            
            #display(history)
            #print(history.val_loss.dtype)
            #print(type(history.val_loss[0]))
            #chosen_idx=history[['val_loss']].idxmin()[0]

            #mlflow.log_metric('best_train_loss',history.at[chosen_idx,'train_loss'])
            #mlflow.log_metric('best_val_loss',history.at[chosen_idx,'val_loss'])
            #mlflow.log_metric('epoch',history.at[chosen_idx,'epoch'])
            #mlflow.log_metric('checkpoint',history.at[chosen_idx,'checkpoint'])
            


CPU times: user 619 ms, sys: 673 ms, total: 1.29 s
Wall time: 500 ms


In [12]:


exp_params={#"model": ['Unet', 'Resnet_Unet'],
            "model": ['Resnet_Unet'],
            "rectype": rectype_strings,
            "learning_rate":[('constant', 1e-4)],#[1e-4],#, 'exp', 'plateau'],
            "max_epochs":[500],
            "batch_size": [4],
}


radial_lines=[20,40,60,80,100]

for rl in radial_lines:
    execute(rl, train_loaders, val_loaders, test_loaders,device,exp_params)




Experiment_id: 585268919028592994
Localização dos artefatos: file:///home/jonathan/MRI_unet_reconstruction/mlruns/585268919028592994
Tags: {}
Lifecycle_stage: active
{'batch_size': 4, 'learning_rate': ('constant', 0.0001), 'max_epochs': 500, 'model': 'Resnet_Unet', 'rectype': 'L2'}
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_20lines_L2/']
['L2']
20
./BIRN_dataset/birn_png/
['./BIRN_dataset/birn_pngs_20lines_L2/']
['L2']
20


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
#train_epoch_den

            


: 

In [ ]:
dftst=pd.DataFrame({'a':[1,2,-3],'b':[2,3,4]})

: 

In [ ]:
chosen_idx=dftst[['a','b']].idxmin()[0]

chosen_idx


: 

: 